# SaaSafras 2 Revenue Optimization Case Study

This notebook aims to solve the SaaSafras revenue optimization problem using a scenario-based approach. We will explore various team allocations across three key roles: New Business Acquisition, Account Management, and Customer Support. The goal is to maximize cumulative revenue over a 24-month period by determining the optimal allocation of 20 team members.


In [131]:
import numpy as np
import pandas as pd

# Constants
initial_customers = 1000
initial_csat = 70  # in percentage
initial_churn_rate = 0.10  # 10%
monthly_fee = 100  # baseline fee per customer
csat_increase_per_agent = 1  # CSAT increase per support agent
churn_decrease_factor = 0.15  # 15% decrease in churn rate per CSAT point
organic_growth = 25  # Organic customer acquisition per month
acquisition_per_agent = 5  # New customers per sales agent
revenue_increase_per_month = 0.20  # 20% revenue increase for managed accounts
max_revenue_increase_months = 6  # Max months of revenue compounding

# Team allocation
team_allocation = [
    {'new_business': 10, 'account_management': 5, 'support': 5},
    {'new_business': 10, 'account_management': 5, 'support': 5},
    {'new_business': 10, 'account_management': 5, 'support': 5},
    {'new_business': 10, 'account_management': 5, 'support': 5},
    {'new_business': 10, 'account_management': 5, 'support': 5},
    {'new_business': 10, 'account_management': 5, 'support': 5},
    {'new_business': 6, 'account_management': 10, 'support': 4},
    {'new_business': 6, 'account_management': 10, 'support': 4},
    {'new_business': 6, 'account_management': 10, 'support': 4},
    {'new_business': 6, 'account_management': 10, 'support': 4},
    {'new_business': 6, 'account_management': 10, 'support': 4},
    {'new_business': 6, 'account_management': 10, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
    {'new_business': 4, 'account_management': 12, 'support': 4},
]

# Initialize variables
customers = initial_customers
csat = initial_csat
churn_rate = initial_churn_rate
cumulative_revenue = 0
monthly_revenues = []

# Initialize a list to store the results
results_list = []

# Function to calculate churn rate based on CSAT
def calculate_churn_rate(csat):
    churn_rate = initial_churn_rate
    for _ in range(csat - initial_csat):
        churn_rate *= (1 - churn_decrease_factor)
    return churn_rate

# Simulation for 24 months
for month, allocation in enumerate(team_allocation, start=1):
    # Calculate new customers acquired
    new_customers = organic_growth + allocation['new_business'] * acquisition_per_agent

    # Update CSAT and calculate churn rate
    csat = initial_csat + allocation['support'] * csat_increase_per_agent
    churn_rate = calculate_churn_rate(csat)

    # Calculate churned customers
    churned_customers = customers * churn_rate

    # Update total customer base
    customers = customers - churned_customers + new_customers

    # Calculate managed and unmanaged customers
    managed_customers = min(allocation['account_management'] * 25, customers)
    unmanaged_customers = customers - managed_customers

    # Revenue calculation for managed customers
    managed_revenue = 0
    for i in range(1, min(month, max_revenue_increase_months) + 1):
        managed_revenue += managed_customers * monthly_fee * (1 + revenue_increase_per_month) ** i
    
    # Calculate total revenue for the month
    monthly_revenue = managed_revenue + unmanaged_customers * monthly_fee
    cumulative_revenue += monthly_revenue
    monthly_revenues.append(monthly_revenue)

    # Store results in the list
    results_list.append({
        'Month': month,
        'New_Customers': new_customers,
        'Churned_Customers': churned_customers,
        'Total_Customers': customers,
        'CSAT': csat,
        'Churn_Rate': churn_rate,
        'Managed_Customers': managed_customers,
        'Unmanaged_Customers': unmanaged_customers,
        'Monthly_Revenue': monthly_revenue,
        'Cumulative_Revenue': cumulative_revenue
    })

# Convert the results list to a DataFrame
results_df = pd.DataFrame(results_list)

# Display the DataFrame
results_df.head()

,Month,New_Customers,Churned_Customers,Total_Customers,CSAT,Churn_Rate,Managed_Customers,Unmanaged_Customers,Monthly_Revenue,Cumulative_Revenue
0,1,75,44.370531,1030.629469,75,0.044371,125,905.629469,105562.946875,105562.946875
1,2,75,45.729577,1059.899892,75,0.044371,125,934.899892,126489.989170,232052.936045
2,3,75,47.028321,1087.871570,75,0.044371,125,962.871570,150887.157043,382940.093088
3,4,75,48.269440,1114.602131,75,0.044371,125,989.602131,179480.213092,562420.306180
4,5,75,49.455489,1140.146642,75,0.044371,125,1015.146642,213138.664224,775558.970404


In [133]:
# Round 'Churned_Customers', 'Total_Customers', and 'Unmanaged_Customers' to whole numbers
results_df['Churned_Customers'] = results_df['Churned_Customers'].round().astype(int)
results_df['Total_Customers'] = results_df['Total_Customers'].round().astype(int)
results_df['Unmanaged_Customers'] = results_df['Unmanaged_Customers'].round().astype(int)

# Round 'Monthly_Revenue' and 'Cumulative_Revenue' to 2 decimal places
results_df['Monthly_Revenue'] = results_df['Monthly_Revenue'].round(2)
results_df['Cumulative_Revenue'] = results_df['Cumulative_Revenue'].round(2)

# Convert 'Monthly_Revenue' and 'Cumulative_Revenue' to currency format
results_df['Monthly_Revenue'] = results_df['Monthly_Revenue'].apply(lambda x: f"${x:,.2f}")
results_df['Cumulative_Revenue'] = results_df['Cumulative_Revenue'].apply(lambda x: f"${x:,.2f}")

# Display the cleaned DataFrame
results_df

,Month,New_Customers,Churned_Customers,Total_Customers,CSAT,Churn_Rate,Managed_Customers,Unmanaged_Customers,Monthly_Revenue,Cumulative_Revenue
0,1,75,44,1031,75,0.044371,125,906,"$105,562.95","$105,562.95"
1,2,75,46,1060,75,0.044371,125,935,"$126,489.99","$232,052.94"
2,3,75,47,1088,75,0.044371,125,963,"$150,887.16","$382,940.09"
3,4,75,48,1115,75,0.044371,125,990,"$179,480.21","$562,420.31"
4,5,75,49,1140,75,0.044371,125,1015,"$213,138.66","$775,558.97"
5,6,75,51,1165,75,0.044371,125,1040,"$252,904.57","$1,028,463.54"
6,7,55,61,1159,74,0.052201,250,909,"$388,774.31","$1,417,237.85"
7,8,55,60,1153,74,0.052201,250,903,"$388,225.47","$1,805,463.32"
8,9,55,60,1148,74,0.052201,250,898,"$387,705.29","$2,193,168.61"
9,10,55,60,1143,74,0.052201,250,893,"$387,212.25","$2,580,380.86"


In [ ]:
# Save the DataFrame to a CSV file
csv_filename = "SaaSafras_Revenue_Data.csv"
results_df.to_csv(csv_filename, index=False)
print(f"Results saved to {csv_filename}")